In [8]:
from time import sleep
import pyautogui
from os import path
from random import uniform, seed
import math

In [2]:
seed()

In [53]:
ANIM_SECONDS = 0.01
CUR_DIR = "C:\\Users\\Shevis\\src\\local\\splitbuilder"
BUILD_MODE = "move_speed.png"
OBJECTS_MENU = "objects_menu.png"
TOOLS_MENU = "tools_menu.png"
MOVEMENT_MENU = "movement_menu.png"
ROTATION_MENU = "rotation_menu.png"
BLOCKS_MENU = "blocks_menu.png"
RAMPS_MENU = "blocks_ramps_menu.png"
GLASS_RAMP = "glass_ramp.png"

In [28]:
X_POS = (216, 935)
Y_POS = (216, 1107)
Z_POS = (216, 1280)

# Start Game

In [46]:
def find_icon(icon_name, no_print=False):
    # Find the game icon
    image_path = path.join(CUR_DIR, 'images', icon_name)
    try:
        game_icon = pyautogui.locateOnScreen(image_path, confidence=0.9)
        if game_icon:
            if not no_print:
                print(f"{icon_name} found: {game_icon}")
            return pyautogui.center(game_icon)
        return None
    except pyautogui.ImageNotFoundException:
        if not no_print:
            print("Unable to locate image file")
        return None

def click_icon(icon_name, no_print=False):
    game_icon = find_icon(icon_name=icon_name, no_print=no_print)
    if game_icon:
        # Start the game
        pyautogui.moveTo(game_icon.x, game_icon.y, duration=ANIM_SECONDS)
        pyautogui.click(game_icon)
    elif not no_print:
        print("Unable to locate game icon", icon_name)

def click_spot(x, y, dur=ANIM_SECONDS):
    pyautogui.moveTo(x, y, duration=dur)
    pyautogui.click(x=x, y=y)

# Movement

In [10]:
def walk_wasd(dur=2):
    pyautogui.keyDown('w')
    sleep(dur)
    pyautogui.keyUp('w')

# Building

In [97]:
def is_active(img):
    icon_path = path.join(CUR_DIR, 'images', img)
    icon = pyautogui.locateOnScreen(icon_path, confidence=0.9)
    return icon is not None

def activate_build_mode():
    if not is_active(BUILD_MODE):
        pyautogui.press('`')

def open_build_menu():
    if not is_active(OBJECTS_MENU):
        pyautogui.press('m')

def enter_value(spot, val):
    click_spot(spot[0], spot[1])
    val_chars = list(str(val))
    for ch in val_chars:
        pyautogui.press(ch)
    pyautogui.press('enter')

def place_object(nav, loc, rot):
    click_icon(OBJECTS_MENU)
    for menu in nav:
        click_icon(menu, no_print=True)
    click_icon(TOOLS_MENU)
    click_icon(MOVEMENT_MENU)
    enter_value(X_POS, loc[0])
    enter_value(Y_POS, loc[1])
    enter_value(Z_POS, loc[2])
    click_icon(ROTATION_MENU)
    enter_value(X_POS, rot[0])
    enter_value(Y_POS, rot[1])
    enter_value(Z_POS, rot[2])
    click_spot(1122, 1122)

def straight_panal_extrapolator(start_loc, start_rot, n_panals):
    pos_x = start_loc[0]
    pos_y = start_loc[1]
    pos_z = start_loc[2]
    rot_x = start_rot[0]
    rot_y = start_rot[1]
    rot_z = start_rot[2]
    panal_width = 400
    sign = math.copysign(1, n_panals)
    panal_mag = abs(n_panals)
    def extrapolate():
        nonlocal pos_x, pos_z
        for _ in range(panal_mag - 1):
            pos_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y)) * panal_width * sign)
            pos_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y)) * panal_width * sign)
            yield (pos_x, pos_y, pos_z), (rot_x, rot_y, rot_z)
    return extrapolate

def place_surf_straight(start_loc, start_rot, n_panals):
    place_object(
        [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP],
        start_loc,
        start_rot
    )
    spe = straight_panal_extrapolator(start_loc=start_loc, start_rot=start_rot, n_panals=n_panals)
    for loc, rot in spe():
        place_object(
            [GLASS_RAMP],
            loc,
            rot
        )
    

# Runner

In [99]:
click_icon('game_icon.png')
activate_build_mode()
open_build_menu()
place_surf_straight(
    (-1546, 10890, 1235),
    (0, -14, -31),
    5
)

game_icon.png found: Box(left=1582, top=2115, width=30, height=20)
objects_menu.png found: Box(left=119, top=372, width=144, height=24)
tools_menu.png found: Box(left=557, top=372, width=104, height=24)
movement_menu.png found: Box(left=401, top=573, width=142, height=20)
rotation_menu.png found: Box(left=628, top=573, width=125, height=20)
objects_menu.png found: Box(left=315, top=372, width=144, height=24)
tools_menu.png found: Box(left=557, top=372, width=104, height=24)
movement_menu.png found: Box(left=401, top=573, width=142, height=20)
rotation_menu.png found: Box(left=628, top=573, width=125, height=20)
objects_menu.png found: Box(left=315, top=372, width=144, height=24)
tools_menu.png found: Box(left=557, top=372, width=104, height=24)
movement_menu.png found: Box(left=401, top=573, width=142, height=20)
rotation_menu.png found: Box(left=628, top=573, width=125, height=20)
objects_menu.png found: Box(left=315, top=372, width=144, height=24)
tools_menu.png found: Box(left=557, 

In [26]:
image = pyautogui.screenshot()
image.save('testing.png')

In [94]:
ep = straight_panal_extrapolator((0, 0, 0), (0, 0, 0), 4)

In [95]:
for p, r in ep():
    print(p, r)

(400, 0, 0) (0, 0, 0)
(800, 0, 0) (0, 0, 0)
(1200, 0, 0) (0, 0, 0)
